<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/R2_Hate_BR_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing needed libraries
!pip install -U scikit-learn
!pip install -U spacy
!python -m spacy download pt
!pip install wandb
from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter
import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import logging
import numpy as np
import sklearn
import pickle
import spacy
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline
from sklearn.model_selection import train_test_split
!pip install simpletransformers "transformers==4.30.2"
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs


BERT IMBAU


In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# base_hatebr = pd.read_csv('/content/drive/MyDrive/PIC /Dataset/HateBR.csv')
# base_hatebr_train_data = base_hatebr.iloc[:,[0, 1]]
# train,eva = train_test_split(base_hatebr_train_data,test_size = 0.2)


train_df = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/HateBR/Sem pré processamento/X_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/HateBR/Sem pré processamento/y_train.csv')
train_df['offensive_language'] =  y_train['offensive_language']
train_df = train_df.iloc[:,[1, 2]]

eva_df = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/HateBR/Sem pré processamento/X_test.csv')
y_test = pd.read_csv('/content/drive/MyDrive/PIC /Teste Treinamento/HateBR/Sem pré processamento/y_test.csv')
eva_df['offensive_language'] =  y_test['offensive_language']
eva_df = eva_df.iloc[:,[1, 2]]


train_df = pd.DataFrame({
     'text': train_df['instagram_comments'],
     'label': train_df['offensive_language']
 })

eva_df = pd.DataFrame({
     'text': eva_df['instagram_comments'],
     'label': eva_df['offensive_language']
 })

model_args = ClassificationArgs(num_train_epochs=1)

model = ClassificationModel(
    "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
    use_cuda=False
)

model.train_model(train_df)

with open('Hate_BR-BERT', 'wb') as file:
    pickle.dump(model, file)

Código para carregar modelo

In [ ]:
with open('/content/drive/MyDrive/PIC /Models/Hate_BR-BERT.pk', 'rb') as f:
    model = pickle.load(f)

validação do modelo

In [46]:
result, model_outputs, wrong_predictions = model.eval_model(eva_df)
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eva_df['offensive_language'].tolist()
predicted = lst

In [48]:
print(classification_report(true, predicted))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92       700
           1       0.92      0.92      0.92       700

    accuracy                           0.92      1400
   macro avg       0.92      0.92      0.92      1400
weighted avg       0.92      0.92      0.92      1400



In [49]:
confusion_matrix(true, predicted)

array([[644,  56],
       [ 58, 642]])